<a href="https://colab.research.google.com/github/arara1930/entailment2SNS/blob/main/entailment2SNS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlflow
!pip install transformers
!pip install pytorch_lightning

import random
import glob
from tqdm import tqdm

import torch
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from mlflow import log_metric, log_param, log_artifact
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

!pip install fugashi
!pip install ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00
  Created wheel for databricks-cli: filename=databricks_cli-0.17.7-py3-none-any.whl size=143860 sha256=ece5f56ed8df105df916bb61a46a3d8401702f862011f931624e3b5150bc1256
  Stored in directory: /root/.cache/pip/wheels/ae/63/93/5402c1a09c1868a59d0b05013484e07af97a9d7b3dbd5bd39a
Successfully built databricks-cli
   

In [ ]:
#データの読み込み
df = pd.read_csv("train.tsv", delimiter='\t')
mapping = {
        'neutral': 2,
        'contradiction': 0,
        'entailment': 1
    }
df.entailment_label_Ja = df.entailment_label_Ja.map(mapping)

t1 = df.sentence_A_Ja.values
t2 = df.sentence_B_Ja.values
labels = df.entailment_label_Ja.values

In [ ]:
# 1. BERT Tokenizerを用いて単語分割・IDへ変換
## Tokenizerの準備
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [ ]:
# 最大単語数の確認
max_len = []

# 1文づつ処理
for sent1, sent2 in zip(t1, t2):
    token_words_1 = tokenizer.tokenize(sent1)
    token_words_2 = tokenizer.tokenize(sent2)
    token_words_1.extend(token_words_2)
    # 文章数を取得してリストへ格納
    max_len.append(len(token_words_1))

max_length = max(max_len) +3 # 最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数

max_length = 500

# 最大の値を確認
print('最大単語数: ', max_length)

最大単語数:  500


In [ ]:
dataset_for_loader = []

end_term = "[SEP]"

# 1文づつ処理
for x , y, label in zip(t1, t2, labels):
    #sent= x  + end_term + y

    encoding = tokenizer(
            x,
            y,
            max_length=max_length,
            padding='max_length',
            truncation=True
        )

    encoding['labels'] = label # ラベルを追加
    encoding = { k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)

In [ ]:
# 80%地点のIDを取得
train_size = int(0.8 * len(dataset_for_loader))
val_size = len(dataset_for_loader) - train_size

# データセットを分割
train_dataset, val_dataset = random_split(dataset_for_loader, [train_size, val_size])

In [ ]:
# データローダの作成
dataloader_train = DataLoader(
    train_dataset, batch_size=16, shuffle=True
)
dataloader_val = DataLoader(val_dataset, batch_size=16)

In [ ]:
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
bert_sc = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)
bert_sc = bert_sc.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [ ]:
class BertForSequenceClassification_pl(pl.LightningModule):

    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        self.save_hyperparameters()

        #BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels = num_labels
        )

    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss)

    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = (labels_predicted == labels).sum().item()
        accuracy = num_correct / labels.size(0)
        self.log('accuracy', accuracy)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)

In [ ]:
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor = 'val_loss',
    mode = 'min',
    save_top_k = 1,
    save_weights_only = True,
    dirpath  = 'model/'
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'min',
    patience = 10
)

trainer = pl.Trainer(
    accelerator = 'gpu',
    #devices = 2,
    #gpus = [2],
    max_epochs = 5,
    callbacks = [checkpoint, early_stopping]
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
model = BertForSequenceClassification_pl(
    MODEL_NAME, num_labels=3, lr=2e-5
)

trainer.fit(model, dataloader_train, dataloader_val)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
best_model_path = checkpoint.best_model_path # ベストモデルのファイル
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)

ベストモデルのファイル:  /content/model/epoch=1-step=500.ckpt
ベストモデルの検証データに対する損失:  tensor(0.3228, device='cuda:0')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl.load_from_checkpoint(
    best_model_path
)

# Transformers対応のモデルを./model_transformesに保存
model.bert_sc.save_pretrained('drive/My Drive/entailment2SNS')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [ ]:
bert_sc = BertForSequenceClassification.from_pretrained(
    'drive/My Drive/entailment2SNS'
)

bert_sc.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
df = pd.read_csv("testSprit.csv", encoding="shift_jis")
mapping = {
        'n': 2,
        'c': 0,
        'e': 1
    }
df.label = df.label.map(mapping)

t1_test = df.t1.values
t2_test = df.t2.values
labels_test = df.label.values

df

,t1,t2,label
0,なんで供託金と年齢制限が必要か分かる例だよなぁ,立候補できてるんだから-全然防げてないじゃん-やっぱ-供託金不要,0
1,立候補できてるんだから全然防げてないじゃんやっぱ供託金不要,個人的には-無くしたら-遊び半分だったり-宣伝目的の人が増えると思うので-あった方が良いと思...,0
2,子供がした契約なら親が取り消せると思います！,民法5条で、未成年者は-親の同意を得なければ-原則として-契約行為をすることができませんので...,1
3,子供がした契約なら親が取り消せると思います！,親がした契約でも、筋が通ってれば-基本的に-返金して貰えますね。アマゾンの設定はガバガバだけ...,1
4,うちの息子がアレクサに「アンパンマンの歌を流して」って言ったらUnlimitedに登録すれば...,うちも-娘にやられました。Unlimitedになってるの-1年くらい-気づかなかった…,1
5,「男まさりな女」枠のハルヒやアスカみたいなキャラ、令和から見るとかなり女性的なんよな、うまく...,ハルヒやアスカって-「男まさりな女」枠だったんだ……,0
6,「男まさりな女」枠のハルヒやアスカみたいなキャラ、令和から見るとかなり女性的なんよな、うまく...,「気が強い女」であって-「男っぽい女」ではないんだよな-ファッションもビジュアルも男ではない...,0
7,「男まさりな女」枠のハルヒやアスカみたいなキャラ、令和から見るとかなり女性的なんよな、うまく...,そこ-男勝りなん？-ワイの認識相当ズレとったわ,0
8,人の親切に対してわがまま言うなやって思う自分の感覚はズレてるんかな,いや-ズレてないと思うけど-配ってる以上-そこらは絶対に注意しなきゃいけない点だよね,1
9,消費期限だとさすがにマズいけれど賞味期限は人体に影響はないと思う。,別に賞味期限切れでも-体にそんな悪影響ないだろうけど、-今回の問題は-賞味期限を管理してなか...,2


In [ ]:
t2_text_splits=[]

for text in t2_test:
  splitted = text.split("-")
  t2_text_splits.append(splitted)


print(t2_text_splits)

[['立候補できてるんだから', '全然防げてないじゃん', 'やっぱ', '供託金不要'], ['個人的には', '無くしたら', '遊び半分だったり', '宣伝目的の人が増えると思うので', 'あった方が良いと思いますね', '高いから下げるとかは賛成です'], ['民法5条で、未成年者は', '親の同意を得なければ', '原則として', '契約行為をすることができませんので、親の預かり知らぬ所で', '行われた契約行為は', '民法120条に基づいて', '本人や親などが', '取り消すことができます。'], ['親がした契約でも、筋が通ってれば', '基本的に', '返金して貰えますね。アマゾンの設定はガバガバだけど', 'サポートは', 'かなり話が通じる。'], ['うちも', '娘にやられました。Unlimitedになってるの', '1年くらい', '気づかなかった…'], ['ハルヒやアスカって', '「男まさりな女」枠だったんだ……'], ['「気が強い女」であって', '「男っぽい女」ではないんだよな', 'ファッションもビジュアルも男ではないし', 'キャラクターとしても男臭いところはない'], ['そこ', '男勝りなん？', 'ワイの認識相当ズレとったわ'], ['いや', 'ズレてないと思うけど', '配ってる以上', 'そこらは絶対に注意しなきゃいけない点だよね'], ['別に賞味期限切れでも', '体にそんな悪影響ないだろうけど、', '今回の問題は', '賞味期限を管理してなかったことじゃない？']]


In [ ]:
predicted = []
correct_labels = []
wrong = []
i = 0

def find_max_variable(**kwargs):
    max_value = float('-inf')  # 初期値を負の無限大に設定
    max_variable = None  # 初期値をNoneに設定
    for variable, value in kwargs.items():
        if isinstance(value, (int, float)) and value > max_value:
            max_value = value
            max_variable = variable
    return max_variable

for x , y, label in zip(t1_test, t2_text_splits, labels_test):
  n = 0
  c = 0
  e = 0

  for y1 in y:

      print(x)
      print(y1)
      correct_labels.append(label)
      correct = label

      encoding = tokenizer(
              x,
              y1,
              max_length=512,
              padding='max_length',
              truncation=True,
              return_tensors='pt'
          )

      encoding = { k: v.cuda() for k, v in encoding.items() }

      with torch.no_grad():
          output = bert_sc.forward(**encoding)
          scores = output.logits
          print(scores)
          labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist()
          if labels_predicted == 2:
            n += 1
          if labels_predicted == 0:
            c += 1
          if labels_predicted == 1:
            e += 1

      if labels_predicted == correct:
          wrong.append(i)

      i+= 1

  neutlal = n/(n+c+e)
  print(neutlal)
  cont = c/(n+c+e)
  print(cont)
  entail = e/(n+c+e)
  print(entail)
  test = find_max_variable(n=neutlal,c=cont,e=entail)
  print(test)
  if test == "n":
    predicted.append(2)
  if test == "c":
    predicted.append(0)
  if test == "e":
    predicted.append(1)


print(predicted)

なんで供託金と年齢制限が必要か分かる例だよなぁ
立候補できてるんだから
tensor([[-0.7951, -0.7829,  1.6169]], device='cuda:0')
なんで供託金と年齢制限が必要か分かる例だよなぁ
全然防げてないじゃん
tensor([[-0.8515, -1.0006,  1.9097]], device='cuda:0')
なんで供託金と年齢制限が必要か分かる例だよなぁ
やっぱ
tensor([[-0.8619, -0.5240,  1.6798]], device='cuda:0')
なんで供託金と年齢制限が必要か分かる例だよなぁ
供託金不要
tensor([[ 1.2497, -0.8703, -0.0020]], device='cuda:0')
0.75
0.25
0.0
n
立候補できてるんだから全然防げてないじゃんやっぱ供託金不要
個人的には
tensor([[-0.7379, -1.0903,  1.8949]], device='cuda:0')
立候補できてるんだから全然防げてないじゃんやっぱ供託金不要
無くしたら
tensor([[ 0.0498, -1.4230,  1.5361]], device='cuda:0')
立候補できてるんだから全然防げてないじゃんやっぱ供託金不要
遊び半分だったり
tensor([[-0.9117, -1.8612,  2.0601]], device='cuda:0')
立候補できてるんだから全然防げてないじゃんやっぱ供託金不要
宣伝目的の人が増えると思うので
tensor([[-0.9528, -1.4148,  2.0089]], device='cuda:0')
立候補できてるんだから全然防げてないじゃんやっぱ供託金不要
あった方が良いと思いますね
tensor([[-0.6076, -1.4155,  2.1660]], device='cuda:0')
立候補できてるんだから全然防げてないじゃんやっぱ供託金不要
高いから下げるとかは賛成です
tensor([[-1.0931, -1.3617,  2.3590]], device='cuda:0')
1.0
0.0
0.0
n
子供がした契約なら親が取り消せると思います！
民法5条で、未成年者は
tensor([[-

In [ ]:
test_num = len(predicted)
num_correct = 0
tp = 0
fp = 0
fn = 0
tn = 0

for i in range(test_num):
    if predicted[i] == correct_labels[i]:
        num_correct += 1

    if predicted[i] == 1 and correct_labels[i] == 1:
        tp += 1

    if (predicted[i] == 1 and correct_labels[i] == 0) or (predicted[i] == 1 and correct_labels[i] == 2):
        fp += 1

    if (predicted[i] == 0 and correct_labels[i] == 1) or (predicted[i] == 2 and correct_labels[i] == 1):
        fn += 1

    if (predicted[i] == 0 and correct_labels[i] == 1) or (predicted[i] == 2 and correct_labels[i] == 1):
        tn += 1
print(tp)
print(fp)
print(fn)
print(tn)

accuracy = num_correct / test_num
recall = tp /(tp + fn)
#precision = tp /(tp + fp)
#f_value = 2*recall*precision / (precision + recall)
print("accuracy: " + str(accuracy))
print("recall: " + str(recall))
print("precision: " + str(precision))
print("f_value: " + str(f_value))


0
0
4
4
accuracy: 0.1
recall: 0.0


NameError: ignored